On vérifie les bonnes installations de packages 

In [1]:
! pip install neo4j
! pip install sqlite3
! pip install os


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for sqlite3
ERROR: Could not find a version that satisfies the requirement os (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for os


Migration vers neo4j

In [ ]:
import os
import sqlite3
from neo4j import GraphDatabase

# Connexion à SQLite
def connect_to_sqlite(db_name="infractions.db"):
    db_path = os.path.abspath(db_name)
    if not os.path.exists(db_path):
        raise FileNotFoundError(f"Le fichier {db_name} est introuvable.")
    return sqlite3.connect(db_path)

# Connexion à Neo4j
def connect_to_neo4j():
    uri = "bolt://localhost:7687"
    username = "neo4j"
    password = "infractions"  # Remplace par ton mot de passe

    try:
        driver = GraphDatabase.driver(uri, auth=(username, password))
        driver.verify_connectivity()
        print("✅ Connexion à Neo4j réussie.")
        return driver
    except Exception as e:
        print(f"❌ Erreur de connexion à Neo4j : {e}")
        exit(1)

# Fonction pour insérer des nœuds en batch
def create_nodes(tx, label, data_list, batch_size=200):
    for i in range(0, len(data_list), batch_size):
        batch = data_list[i:i+batch_size]
        query = f"""
        UNWIND $data AS row
        MERGE (n:{label} {{ {", ".join([f"{key}: row.{key}" for key in batch[0]])} }})
        """
        tx.run(query, data=batch)

# Fonction pour insérer des relations en batch
def create_relationships(tx, start_label, start_key, end_label, end_key, rel_type, data_list, batch_size=200):
    for i in range(0, len(data_list), batch_size):
        batch = data_list[i:i+batch_size]
        query = f"""
        UNWIND $data AS row
        MATCH (a:{start_label} {{ {start_key}: row.{start_key} }})
        MATCH (b:{end_label} {{ {end_key}: row.{end_key} }})
        MERGE (a)-[:{rel_type}]->(b)
        """
        tx.run(query, data=batch)

# Fonction pour insérer les faits et leurs relations en batchs réduits
def create_fait_relationships(tx, data_list, batch_size=200):
    for i in range(0, len(data_list), batch_size):
        batch = data_list[i:i+batch_size]
        query = """
        UNWIND $data AS row
        MATCH (b:Brigade { id_brigade: row.id_brigade })
        MATCH (i:Infraction { code_infr: row.code_infr })
        MATCH (d:Departement { code_dep: row.code_dep })
        MATCH (a:Annee { annee: row.annee })
        MATCH (fO:ForceOrdre { force_odre: row.force_odre })
        
        MERGE (f:Fait { annee: row.annee, nb_infraction: row.nb_infraction })
        WITH f, b, i, d, a, fO

        MERGE (b)-[:A_ENREGISTRÉ]->(f)
        MERGE (f)-[:CONCERNE]->(i)
        MERGE (f)-[:A_LIEU_DANS]->(d)
        MERGE (i)-[:CONSTATE_PAR]->(fO)
        MERGE (f)-[:SUR_ANNEE]->(a)
        """
        tx.run(query, data=batch)

# Fonction principale de migration
def migrate_data(sqlite_conn, neo4j_driver, batch_size=200):
    sqlite_cursor = sqlite_conn.cursor()
    with neo4j_driver.session() as session:

        # Migration des infractions
        print("➡ Migration des infractions...")
        sqlite_cursor.execute("SELECT code_infr, lib_infr FROM infractions")
        batch = [{"code_infr": row[0], "lib_infr": row[1]} for row in sqlite_cursor.fetchall()]
        session.execute_write(create_nodes, "Infraction", batch, batch_size)
        print("✅ Infractions migrées.")

        # Migration des départements
        print("➡ Migration des départements...")
        sqlite_cursor.execute("SELECT code_dep, nom_dep FROM départements")
        batch = [{"code_dep": row[0], "nom_dep": row[1]} for row in sqlite_cursor.fetchall()]
        session.execute_write(create_nodes, "Departement", batch, batch_size)
        print("✅ Départements migrés.")

        # Migration des forces de l'ordre
        print("➡ Migration des forces de l'ordre...")
        sqlite_cursor.execute("SELECT force_odre FROM force_odre")
        batch = [{"force_odre": row[0]} for row in sqlite_cursor.fetchall()]
        session.execute_write(create_nodes, "ForceOrdre", batch, batch_size)
        print("✅ Forces de l'ordre migrées.")

        # Migration des années
        print("➡ Migration des années...")
        sqlite_cursor.execute("SELECT annee FROM annee")
        batch = [{"annee": row[0]} for row in sqlite_cursor.fetchall()]
        session.execute_write(create_nodes, "Annee", batch, batch_size)
        print("✅ Années migrées.")

        # Migration des brigades
        print("➡ Migration des brigades...")
        sqlite_cursor.execute("SELECT id_brigade, lib_brigade, code_dep, force_odre FROM brigade")
        batch = [{"id_brigade": row[0], "lib_brigade": row[1], "code_dep": row[2], "force_odre": row[3]} for row in sqlite_cursor.fetchall()]
        session.execute_write(create_nodes, "Brigade", batch, batch_size)
        print("✅ Brigades migrées.")

        # Création des relations Brigade ↔ Département
        print("➡ Création des relations Brigade ↔ Département...")
        session.execute_write(create_relationships, "Brigade", "code_dep", "Departement", "code_dep", "APPARTIENT_A", batch, batch_size)
        print("✅ Relations créées.")

        # Création des relations Brigade ↔ Force de l'ordre
        print("➡ Création des relations Brigade ↔ Force de l'ordre...")
        session.execute_write(create_relationships, "Brigade", "force_odre", "ForceOrdre", "force_odre", "FAIT_PARTIE_DE", batch, batch_size)
        print("✅ Relations créées.")

        # Vérification des NULL dans `nb_infraction`
        sqlite_cursor.execute("SELECT COUNT(*) FROM fait WHERE nb_infraction IS NULL")
        null_count = sqlite_cursor.fetchone()[0]
        print(f"⚠️ {null_count} lignes avec `nb_infraction` NULL détectées.")

        # Migration des faits et relations avec batchs réduits
        print("➡ Migration des faits (infractions enregistrées)...")
        sqlite_cursor.execute("""
            SELECT f.code_infr, f.id_brigade, f.annee, 
                   COALESCE(f.nb_infraction, 0) AS nb_infraction,  -- Remplace NULL par 0
                   b.code_dep, b.force_odre
            FROM fait f
            JOIN brigade b ON f.id_brigade = b.id_brigade
        """)
        batch = [{"code_infr": row[0], "id_brigade": row[1], "annee": row[2], 
                  "nb_infraction": row[3], "code_dep": row[4], "force_odre": row[5]} 
                 for row in sqlite_cursor.fetchall()]
        session.execute_write(create_fait_relationships, batch, batch_size)
        print("✅ Faits migrés avec succès.")

# Point d'entrée du script
if __name__ == "__main__":
    sqlite_conn = connect_to_sqlite()
    neo4j_driver = connect_to_neo4j()
    migrate_data(sqlite_conn, neo4j_driver)
    print("🎉 Migration terminée avec succès.")


❌ Erreur de connexion à Neo4j : {code: Neo.ClientError.Security.Unauthorized} {message: The client is unauthorized due to authentication failure.}


AttributeError: 'NoneType' object has no attribute 'session'